# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 12:30PM,253924,16,8910465,Sartorius Bioprocess Solutions,Released
1,Dec 28 2022 12:25PM,253923,10,MSP217062,"Methapharm, Inc.",Released
2,Dec 28 2022 12:25PM,253923,10,MSP217063,"Methapharm, Inc.",Released
3,Dec 28 2022 12:25PM,253923,10,MSP217052,"Methapharm, Inc.",Released
4,Dec 28 2022 12:25PM,253923,10,MSP217053,"Methapharm, Inc.",Released
5,Dec 28 2022 12:25PM,253923,10,MSP217054,"Methapharm, Inc.",Released
6,Dec 28 2022 12:25PM,253923,10,MSP217056,"Methapharm, Inc.",Released
7,Dec 28 2022 12:25PM,253923,10,MSP217057,"Methapharm, Inc.",Released
8,Dec 28 2022 12:25PM,253923,10,MSP217058,"Methapharm, Inc.",Released
9,Dec 28 2022 12:25PM,253923,10,MSP217059,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,253919,Released,1
39,253920,Released,12
40,253922,Released,6
41,253923,Released,10
42,253924,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253919,NaN,NaN,1.0
253920,NaN,NaN,12.0
253922,NaN,NaN,6.0
253923,NaN,NaN,10.0
253924,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253850,0.0,0.0,1.0
253853,0.0,0.0,1.0
253855,0.0,0.0,5.0
253856,4.0,4.0,4.0
253862,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253850,0,0,1
253853,0,0,1
253855,0,0,5
253856,4,4,4
253862,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253850,0,0,1
1,253853,0,0,1
2,253855,0,0,5
3,253856,4,4,4
4,253862,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253850,,,1
1,253853,,,1
2,253855,,,5
3,253856,4,4,4
4,253862,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc."
11,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc."
15,Dec 28 2022 12:21PM,253922,10,Methapharm-G
17,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation
29,Dec 28 2022 12:14PM,253919,17,"Alliance Pharma, Inc."
30,Dec 28 2022 12:10PM,253918,10,Emerginnova
31,Dec 28 2022 12:09PM,253917,10,Emerginnova
32,Dec 28 2022 11:50AM,253916,21,"NBTY Global, Inc."
33,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions,,,1
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc.",,,10
2,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc.",,,6
3,Dec 28 2022 12:21PM,253922,10,Methapharm-G,,,6
4,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation,,,12
5,Dec 28 2022 12:14PM,253919,17,"Alliance Pharma, Inc.",,,1
6,Dec 28 2022 12:10PM,253918,10,Emerginnova,,,1
7,Dec 28 2022 12:09PM,253917,10,Emerginnova,,,1
8,Dec 28 2022 11:50AM,253916,21,"NBTY Global, Inc.",,,1
9,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions,1,,
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc.",10,,
2,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc.",6,,
3,Dec 28 2022 12:21PM,253922,10,Methapharm-G,6,,
4,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation,12,,
5,Dec 28 2022 12:14PM,253919,17,"Alliance Pharma, Inc.",1,,
6,Dec 28 2022 12:10PM,253918,10,Emerginnova,1,,
7,Dec 28 2022 12:09PM,253917,10,Emerginnova,1,,
8,Dec 28 2022 11:50AM,253916,21,"NBTY Global, Inc.",1,,
9,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions,1,,
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc.",10,,
2,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc.",6,,
3,Dec 28 2022 12:21PM,253922,10,Methapharm-G,6,,
4,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc.",10.0,NaN,NaN
2,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc.",6.0,NaN,NaN
3,Dec 28 2022 12:21PM,253922,10,Methapharm-G,6.0,NaN,NaN
4,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 12:30PM,253924,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Dec 28 2022 12:25PM,253923,10,"Methapharm, Inc.",10.0,0.0,0.0
2,Dec 28 2022 12:21PM,253922,10,"Methapharm, Inc.",6.0,0.0,0.0
3,Dec 28 2022 12:21PM,253922,10,Methapharm-G,6.0,0.0,0.0
4,Dec 28 2022 12:15PM,253920,10,ISDIN Corporation,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7870741,262.0,0.0,0.0
16,253924,1.0,0.0,0.0
17,253919,1.0,0.0,0.0
19,2031116,31.0,6.0,4.0
21,253916,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7870741,262.0,0.0,0.0
1,16,253924,1.0,0.0,0.0
2,17,253919,1.0,0.0,0.0
3,19,2031116,31.0,6.0,4.0
4,21,253916,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,262.0,0.0,0.0
1,16,1.0,0.0,0.0
2,17,1.0,0.0,0.0
3,19,31.0,6.0,4.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,262.0
1,16,Released,1.0
2,17,Released,1.0
3,19,Released,31.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,21
Status,,,,,
Completed,0.0,0.0,0.0,4.0,0.0
Executing,0.0,0.0,0.0,6.0,0.0
Released,262.0,1.0,1.0,31.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,21
0,Completed,0.0,0.0,0.0,4.0,0.0
1,Executing,0.0,0.0,0.0,6.0,0.0
2,Released,262.0,1.0,1.0,31.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,21
0,Completed,0.0,0.0,0.0,4.0,0.0
1,Executing,0.0,0.0,0.0,6.0,0.0
2,Released,262.0,1.0,1.0,31.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()